<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting/blob/main/myProspects_Valuation_Gap_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

Run date time (IST): 2024-02-07 11:03:48


In [2]:
!pip install yfinance

In [3]:
import pandas as pd
import numpy as np

mon = 'Dec'

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/'

mypf = pd.read_csv(path_mypf + 'myProspectsScrips.csv')

#mypf = mypf[(mypf['Criteria'] == 'V40') | (mypf['Criteria'] == 'V40N')]
mypf = mypf[mypf['Criteria'].isin(['V40','V40N','VR','AR', 'V200'])]

stock_n100 = mypf['Symbol'].unique()

exclude = ['ACC','AMBUJACEM','CADILAHC','PGHH','SIEMENS', 'GILLETTE', 'NESTLEIND', 'SANOFI', 'AMARAJABAT', 'SCHAEFFLER ']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100[:]
#stock_ids = ['HDFCBANK', 'COLPAL']
print('Stocks to be scanned:',len(stock_ids))

Stocks to be scanned: 134


In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

df_latest_analysis = pd.DataFrame()

for stock_name in stock_ids:


    #print(stock_name)

    link_blueprint = "https://www.screener.in/company/"

    stock_list = ['3MINDIA', 'AUBANK', 'COLPAL', 'GLAND', 'LUXIND', 'WHIRLPOOL' , 'IEX', 'ROSSARI', 'VALIANTORG', 'VINATIORGA', 'TATAELXSI',
                  'AMARAJABAT', 'EQUITAS', 'AAVAS', 'CAMS', 'DREAMFOLKS', 'LICI']

    if stock_name in stock_list:
        actual_link = link_blueprint + stock_name.upper()
    else:
        actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"

    def get_list(actual_link):
        page = requests.get(actual_link)
        link_soup = BeautifulSoup(page.content, 'html.parser')

        test_list = link_soup.find(id="profit-loss").text.replace("  ", "").split('\n')

        while("" in test_list) :
            test_list.remove("")
        while("\r" in test_list) :
            test_list.remove("\r")
        while(" \r" in test_list) :
            test_list.remove(" \r")

        return test_list
    #print(actual_link)
    test_list = get_list(actual_link)

    #print(len(test_list))
    n = len(test_list)

    if n < 100:
        actual_link = link_blueprint + stock_name.upper()
        test_list = get_list(actual_link)
        #print(len(test_list))

    #print(test_list)
    if 'TTM' not in test_list:
        end_year = 'Mar 2023'
    else:
        end_year = 'TTM'

        df_yoy = pd.DataFrame()
        #print(test_list)
        s = test_list.index(end_year) +1 - test_list.index('Mar 2018')

        df_yoy['Year'] = test_list[test_list.index('Mar 2018'):test_list.index(end_year)+1]
        df_yoy['Sales'] = test_list[test_list.index('Expenses\xa0+')-s:test_list.index('Expenses\xa0+')]
        df_yoy['Profit'] = test_list[test_list.index('Tax %')-s:test_list.index('Tax %')]

        df_yoy.set_index('Year', inplace=True)
        df_yoy = df_yoy.T
        df_yoy['Stock'] = stock_name.upper() + '.NS'
        #df_yoy
        df_latest_analysis = pd.concat([df_latest_analysis, df_yoy])

df_latest_analysis.index.names = ['Type']
df_latest_analysis.reset_index(inplace=True)
df_latest_analysis.rename(columns={'TTM': mon+' 2023'}, inplace=True)
#df_latest_analysis.drop('TTM', axis=1, inplace=True)
cols = [ 'Mar 2018', 'Mar 2019', 'Mar 2020', 'Mar 2021', 'Mar 2022', 'Mar 2023', mon+' 2023']
for col in cols:
    df_latest_analysis[col] = df_latest_analysis[col].str.replace(',', '').astype(float)

In [5]:
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()

stock_df_final = pd.DataFrame()

for stock_name in stock_ids:

  stock_symbol = stock_name + '.NS'
  short_window = 20
  mid_window = 50
  long_window = 200
  moving_avg = 'SMA'

  # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
  stock_df = yfin.Ticker(stock_symbol).history(period='6y', interval='1mo')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
  stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
  stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
  stock_df.dropna(axis = 0, inplace = True) # remove any null rows

  stock_df['Stock'] = stock_symbol

  stock_df['Close'] = round(stock_df['Close'],2)
  stock_df['Mon'] = stock_df.index.month_name().str.slice(stop=3)
  stock_df['Year'] = stock_df.index.strftime('%Y')
  stock_df['Year'] = stock_df['Mon'] + ' ' + stock_df['Year']
  cols = [ 'Year', 'Close']
  stock_df = stock_df[cols]
  stock_df = stock_df[(stock_df['Year'].str.contains("Mar")) | (stock_df['Year'].str.contains(mon+' 2023'))]
  stock_df = stock_df.set_index('Year')
  stock_df = stock_df.T
  stock_df['Stock'] = stock_symbol
  stock_df.index.names = ['Type']
  stock_df = stock_df.reset_index()
  stock_df_final = pd.concat([stock_df_final, stock_df])

stock_df_final

Year,Type,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Dec 2023,Stock
0,Close,18667.90,23329.87,18105.45,29268.68,18941.44,22882.53,37309.15,3MINDIA.NS
0,Close,203.15,144.82,113.15,249.70,344.70,283.00,566.25,5PAISA.NS
0,Close,272.25,375.83,378.50,651.85,950.19,516.46,649.60,AARTIIND.NS
0,Close,NaN,1155.80,1188.85,2420.85,2569.45,1610.75,1532.25,AAVAS.NS
0,Close,5202.22,6973.80,14775.46,14573.12,17372.22,21929.65,22851.25,ABBOTINDIA.NS
...,...,...,...,...,...,...,...,...,...
0,Close,308.00,470.79,236.75,349.14,735.10,570.60,595.46,VIPIND.NS
0,Close,NaN,NaN,321.81,458.00,477.58,680.83,816.85,WESTLIFE.NS
0,Close,1481.61,1489.94,1794.54,2207.28,1562.80,1310.90,1366.85,WHIRLPOOL.NS
0,Close,205.94,249.56,193.43,408.18,584.32,364.47,470.30,WIPRO.NS


In [6]:
df_gap_analysis = df_latest_analysis.append(stock_df_final)
df_gap_analysis['Growth%'] = (df_gap_analysis[mon+' 2023'] - df_gap_analysis['Mar 2018'])*100/df_gap_analysis['Mar 2018']
df_gap_analysis['Growth%'] = round(df_gap_analysis['Growth%'],2)
df_gap_analysis

Year,Type,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Dec 2023,Stock,Growth%
0,Sales,2556.00,2809.00,2765.00,2420.00,3078.00,3733.00,3871.00,3MINDIA.NS,51.45
1,Profit,480.00,497.00,320.00,202.00,360.00,561.00,655.00,3MINDIA.NS,36.46
2,Sales,20.00,61.00,108.00,195.00,298.00,338.00,372.00,5PAISA.NS,1760.00
3,Profit,-33.00,-25.00,-10.00,20.00,18.00,58.00,84.00,5PAISA.NS,-354.55
4,Sales,3806.00,4168.00,4186.00,4506.00,6086.00,6619.00,6191.00,AARTIIND.NS,62.66
...,...,...,...,...,...,...,...,...,...,...
0,Close,308.00,470.79,236.75,349.14,735.10,570.60,595.46,VIPIND.NS,93.33
0,Close,NaN,NaN,321.81,458.00,477.58,680.83,816.85,WESTLIFE.NS,NaN
0,Close,1481.61,1489.94,1794.54,2207.28,1562.80,1310.90,1366.85,WHIRLPOOL.NS,-7.75
0,Close,205.94,249.56,193.43,408.18,584.32,364.47,470.30,WIPRO.NS,128.37


In [7]:
stock_df_gap = pd.DataFrame()

for stock_name in stock_ids:

  stock_symbol = stock_name + '.NS'
  #print(stock_symbol)
  tmp_df = df_gap_analysis[df_gap_analysis['Stock'] == stock_symbol]
  try:
    gap_perc = round(tmp_df[tmp_df['Type'] == 'Profit']['Growth%'].values[0]  - tmp_df[tmp_df['Type'] == 'Close']['Growth%'].values[0],2)
  except:
    pass
  #print(gap_perc)
  tmp_df = pd.DataFrame()
  tmp_df['Stock'] = [stock_symbol]
  tmp_df['Gap%'] = [gap_perc]
  stock_df_gap = pd.concat([stock_df_gap, tmp_df])

cols = ['Stock', 'Criteria']
tmp_df = mypf[cols]
stock_df_gap = pd.merge(stock_df_gap, tmp_df,  on='Stock')
stock_df_gap.sort_values(['Gap%'], ascending=[False]).head(30)

,Stock,Gap%,Criteria
13,AXISBANK.NS,3677.25,V40
117,TATACOMM.NS,2522.68,AR
71,J&KBANK.NS,1599.03,VR
116,TANLA.NS,1470.98,AR
133,WSTCSTPAPR.NS,354.77,VR
59,ICICIBANK.NS,167.44,V40
90,MOTILALOFS.NS,151.43,V40N
65,INFIBEAM.NS,136.39,AR
64,INDUSINDBK.NS,118.84,VR
22,BAYERCROP.NS,108.42,V40N


In [8]:
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    #start = datetime.datetime(*map(int, start_date.split('-')))
    #end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol).history(period='6y', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    #stock_df.columns = ['Close']

    #stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Stock'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Max'] = round(max(stock_df['Close']),2)
    #stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start1_date)]
    #stock_df['Min'] = round(min(stock_df['Close']),2)
    #stock_df['Gained%'] = round((stock_df['Close'] - stock_df['Min'])*100/stock_df['Min'],2)
    #stock_df.ta.rsi(append=True)
    #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    #stock_df['Close'] = round(stock_df['Close'],0)
    #stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['ATH%'] = round((stock_df['Max'] - stock_df['Close'])*100/stock_df['Close'],2)
    #stock_df['Sector'] = stock_symbol
    stock_df = stock_df.tail(1)
    cols = ['Stock', 'ATH%']
    return stock_df[cols]

In [9]:
stock_df_max = pd.DataFrame()
for stock_name in stock_ids:

  stock_symbol = stock_name + '.NS'
  tmp_df = stock_prec_dev(stock_symbol)
  stock_df_max = pd.concat([stock_df_max, tmp_df])
stock_df_max

,Stock,ATH%
Date,,
2024-02-07 00:00:00+05:30,3MINDIA.NS,10.38
2024-02-07 00:00:00+05:30,5PAISA.NS,21.92
2024-02-07 00:00:00+05:30,AARTIIND.NS,73.60
2024-02-07 00:00:00+05:30,AAVAS.NS,120.71
2024-02-07 00:00:00+05:30,ABBOTINDIA.NS,0.00
...,...,...
2024-02-07 00:00:00+05:30,VIPIND.NS,35.29
2024-02-07 00:00:00+05:30,WESTLIFE.NS,21.02
2024-02-07 00:00:00+05:30,WHIRLPOOL.NS,93.72


In [10]:
stock_df_final = pd.merge(stock_df_gap, stock_df_max,  on='Stock')
stock_df_final[(stock_df_final['Gap%'] >=50) & (stock_df_final['Gap%'] <= 300)].sort_values(by = 'ATH%', ascending=False)

,Stock,Gap%,Criteria,ATH%
65,INFIBEAM.NS,136.39,AR,171.53
36,CUB.NS,50.55,AR,81.00
111,SIS.NS,100.12,V40N,39.74
77,KANSAINER.NS,90.04,V40N,33.63
64,INDUSINDBK.NS,118.84,VR,30.79
25,BLUESTARCO.NS,58.59,AR,29.35
18,BAJFINANCE.NS,57.74,V40,22.63
78,KOTAKBANK.NS,69.34,V40,22.35
55,HDFCBANK.NS,76.53,V40,20.13
43,EMAMILTD.NS,70.83,V200,16.45


In [11]:
df_gap_analysis[df_gap_analysis['Stock']  == 'TATACOMM.NS']

Year,Type,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Dec 2023,Stock,Growth%
228,Sales,16772.00,16525.0,17068.00,17100.00,16725.00,17838.00,19846.0,TATACOMM.NS,18.33
229,Profit,46.00,343.0,140.00,1609.00,2000.00,2063.00,1301.0,TATACOMM.NS,2728.26
0,Close,579.32,576.9,221.64,1014.82,1185.89,1228.74,1770.3,TATACOMM.NS,205.58
